# Introduction to Docker

**Learning Objectives**
  * Build and run Docker containers
  * Pull Docker images from Docker Hub and Google Container Registry
  * Push Docker images to Google Container Registry

## Overview

Docker is an open platform for developing, shipping, and running applications. With Docker, you can separate your applications from your infrastructure and treat your infrastructure like a managed application. Docker helps you ship code faster, test faster, deploy faster, and shorten the cycle between writing code and running code.

Docker does this by combining kernel containerization features with workflows and tooling that helps you manage and deploy your applications.

Docker containers can be directly used in Kubernetes, which allows them to be run in the Kubernetes Engine with ease. After learning the essentials of Docker, you will have the skillset to start developing Kubernetes and containerized applications.

## Basic Docker commands

See what docker images you have. 

In [1]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED        SIZE
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   8 months ago   1.73GB


If this is the first time working with docker you won't have any repositories listed. 

**Note**. If you are running this in an AI Notebook, then you should see a single image `gcr.io/inverting-proxy/agent`. This is the container that is currently running the AI Notebook. 

Let's use `docker run` to pull a docker image called `hello-world` from the public registry. The docker daemon will search for the `hello-world` image, if it doesn't find the image locally, it pulls the image from a public registry called Docker Hub, creates a container from that image, and runs the container for you.

In [2]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

Digest: sha256:37a0b92b08d4919615c3ee023f7ddb068d12b8387475d64c622ac30f45c29c51
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https

Now when we look at our docker images we should see `hello-world` there as well.

In [3]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED        SIZE
hello-world                    latest    feb5d9fea6a5   6 weeks ago    13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   8 months ago   1.73GB


This is the image pulled from the Docker Hub public registry. The Image ID is in `SHA256` hash format—this field specifies the Docker image that's been provisioned. When the docker daemon can't find an image locally, it will by default search the public registry for the image. Let's run the container again:

Now, if we want to run `docker run hello-world` again, it won't have to download from the container registry.

To see all docker containers running, use `docker ps`.

In [4]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED      STATUS      PORTS     NAMES
ae392520191d   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   6 days ago   Up 6 days             proxy-agent


There are no running containers. **Note. If you are running this in at AI Notebook, you'll see one container running.**

The `hello-world` containers you ran previously already exited. In order to see all containers, including ones that have finished executing, run docker `ps -a`:

In [5]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED         STATUS                     PORTS     NAMES
144673fb34f0   hello-world                    "/hello"                 7 seconds ago   Exited (0) 6 seconds ago             priceless_dhawan
ae392520191d   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   6 days ago      Up 6 days                            proxy-agent


This shows you the Container ID, a UUID generated by Docker to identify the container, and more metadata about the run. The container Names are also randomly generated but can be specified with docker run --name [container-name] hello-world.

## Build a Docker container

Let's build a Docker image that's based on a simple node application.

**Exercise**

Open the text file called `intro.docker` in the `dockerfiles` folder and complete the TODO there. 

Your dockerfile should have the following steps

 1. use `FROM` to inherit an official Node runtime as the parent image; e.g. node:6
 2. use `WORKDIR` to seet the working directory to /app
 3. use `ADD` to copy the current directory to the container at /app
 4. use `EXPOSE` to make the containers port 80 available to the outside world
 5. use `CMD` to run the command `node ./src/app.js`

This file instructs the Docker daemon on how to build your image.

The initial line specifies the base parent image, which in this case is the official Docker image for node version 6.
In the second, we set the working (current) directory of the container.
In the third, we add the current directory's contents (indicated by the "." ) into the container.
Then we expose the container's port so it can accept connections on that port and finally run the node command to start the application.

Check out the other [Docker command references](https://docs.docker.com/engine/reference/builder/#known-issues-run) to understand what each line does.

We're going to use this Docker container to run a simple node.js app. Have a look at `app.js`. This is a simple HTTP server that listens on port 80 and returns "Hello World."


Now let's build the image. Note again the "`.`", which means current directory so you need to run this command from within the directory that has the Dockerfile.

The `-t` is to name and tag an image with the `name:tag` syntax. The name of the image is `node-app` and the tag is `0.1`. The tag is highly recommended when building Docker images. If you don't specify a tag, the tag will default to latest and it becomes more difficult to distinguish newer images from older ones. Also notice how each line in the Dockerfile above results in intermediate container layers as the image is built.

**Exercise**

Use `docker build` to build the docker image at `dockerfiles/intro.docker`. Tag the image `node-app:0.1`. 

In [6]:
!pwd

/home/jupyter/asl-ml-immersion/notebooks/docker_and_kubernetes/labs


In [32]:
%%bash
cd /home/jupyter/asl-ml-immersion/notebooks/docker_and_kubernetes/labs/dockerfiles
docker build -t node-app:0.1 . -f intro.docker

Sending build context to Docker daemon  7.168kB
Step 1/5 : FROM node:latest
 ---> 7220633f01cd
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> f1a73ad8418b
Step 3/5 : COPY src/ /app
 ---> ce4147ef958b
Step 4/5 : EXPOSE 80
 ---> Running in 4d87d712f1fe
Removing intermediate container 4d87d712f1fe
 ---> a0100313e822
Step 5/5 : ENTRYPOINT node app.js
 ---> Running in 0bb2bf17dfba
Removing intermediate container 0bb2bf17dfba
 ---> e0ad8979a21e
Successfully built e0ad8979a21e
Successfully tagged node-app:0.1


Let's check that the image has been created correctly. 

In [33]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED         SIZE
node-app                       0.1       e0ad8979a21e   3 seconds ago   992MB
node                           latest    7220633f01cd   2 weeks ago     992MB
hello-world                    latest    feb5d9fea6a5   6 weeks ago     13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   8 months ago    1.73GB


You should see a `node-app` repository that was created only seconds ago. 

Notice `node` is the base image and `node-app` is the image you built. You can't remove `node` without removing `node-app` first. The size of the image is relatively small compared to VMs. Other versions of the node image such as `node:slim` and `node:alpine` can give you even smaller images for easier portability. The topic of slimming down container sizes is further explored in Advanced Topics. You can view all versions in the official repository here.

Note, you can remove an image from your docker images using `docker rmi [repository]:[tag]`.

## Run a Docker container

Now we'll run the container based on the image you built above using the `docker run` command. The `--name` flag allows you to name the container if you like. And `-p` instructs Docker to map the host's port 4000 to the container's port 80. This allows you to reach the server at http://localhost:4000. Without port mapping, you would not be able to reach the container at localhost.

In [34]:
!docker rmi 'node-app'

Error: No such image: a50c890deb6e


In [38]:
%%bash
docker stop 'my-app'
docker rm 'my-app'

my-app
my-app


**Exercise**

Use `docker run` to run the container you just build called `node-app:0.1`. Assign the host port `4000` to port `80` and assign it the name `my-app`.

In [40]:
%%bash
docker run -d -p 4000:80 --name 'my-app' 'node-app:0.1'

430e25a104676a4aba47f2bb2154438027fcc40cc1c9b6e3cac1cf58ae7e0a42


To test out the server, open a terminal window and type the following command:

```bash
curl http://localhost:4000
```

You should see the server respond with `Hello World`

The container will run as long as the initial terminal is running. If you want to stop the container, run the following command in the terminal to stop and remove the container:

```bash
docker stop my-app && docker rm my-app
```
After a few moments the container will stop. You should notice the cell above will complete execution.

#### Running the container in the background
If you want to the container to run in the background (not tied to the terminal's session), you need to specify the `-d` flag.
Now run the following command to start the container in the background

**Exercise**

Modify your command above with `-d` flag to run `my-app` in the background.

In [ ]:
%%bash
TODO: Your code goes here

Your container is now running in the background. You can check the status of your running container using `docker ps`

In [41]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS                  NAMES
430e25a10467   node-app:0.1                   "/bin/sh -c 'node ap…"   32 seconds ago   Up 31 seconds   0.0.0.0:4000->80/tcp   my-app
ae392520191d   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   6 days ago       Up 6 days                              proxy-agent


Notice the container is running in the output of docker ps. You can look at the logs by executing `docker logs [container_id]`. 

In [42]:
# Note, your container id will be different
!docker logs 430e25a10467

Server running at http://0.0.0.0:80/


You should see 
```bash
Server running at http://0.0.0.0:80/
```
If you want to follow the log's output as the container is running, use the `-f` option.

## Modify & Publish

Let's modify the application and push it to your Google Cloud Repository (gcr). After that you'll remove all local containers and images to simulate a fresh environment, and then pull and run your containers from gcr. This will demonstrate the portability of Docker containers.

### Edit `app.js`
Open the file `./src/app.js` with the text editor and replace "Hello World" with another string. Then build this new image. 

**Exercise**

After modifying the `app.js` file, use `docker build` to build a new container called `node-app:0.2` from the same docker file. 

In [44]:
%%bash
cd /home/jupyter/asl-ml-immersion/notebooks/docker_and_kubernetes/labs/dockerfiles
docker build -t node-app:0.2 . -f intro.docker

Sending build context to Docker daemon  8.704kB
Step 1/5 : FROM node:latest
 ---> 7220633f01cd
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> f1a73ad8418b
Step 3/5 : COPY src/ /app
 ---> d304bd771638
Step 4/5 : EXPOSE 80
 ---> Running in b623fc1d3e5a
Removing intermediate container b623fc1d3e5a
 ---> 5f2e33a18e63
Step 5/5 : ENTRYPOINT node app.js
 ---> Running in ff489ccc6ad3
Removing intermediate container ff489ccc6ad3
 ---> b13446ebdebc
Successfully built b13446ebdebc
Successfully tagged node-app:0.2


Notice in `Step 2` of the output we are using an existing cache layer. From `Step 3` and on, the layers are modified because we made a change in `app.js`.

Run another container with the new image version. Notice how we map the host's port 8000 instead of 80. We can't use host port 4000 because it's already in use. 

**Exercise**

Run this new container in the background using a different port and with the name `my-app-2`.

In [45]:
%%bash
docker run -d -p 4010:80 --name 'my-app-2' 'node-app:0.2'

2e174c986158a5d5ef289ee2b77ced1535a4aa929a2062fd8ef32c1b3a86dec2


You can check that both container are running using `docker ps`.

In [46]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED         STATUS         PORTS                  NAMES
2e174c986158   node-app:0.2                   "/bin/sh -c 'node ap…"   3 seconds ago   Up 2 seconds   0.0.0.0:4010->80/tcp   my-app-2
430e25a10467   node-app:0.1                   "/bin/sh -c 'node ap…"   5 minutes ago   Up 5 minutes   0.0.0.0:4000->80/tcp   my-app
ae392520191d   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   6 days ago      Up 6 days                             proxy-agent


And let's test boht containers using `curl` as before:

In [47]:
!curl http://localhost:4010

Hello World!


In [48]:
!curl http://localhost:4000

Hello Wolrd!


Recall, to stop a container running, you can execute the following command either in a terminal or (because they are running in the background) in a cell in this notebook. 

### Publish to gcr

Now you're going to push your image to the Google Container Registry (gcr). To push images to your private registry hosted by gcr, you need to tag the images with a registry name. The format is `[hostname]/[project-id]/[image]:[tag]`.

For gcr:

  * `[hostname]`= gcr.io
  * `[project-id]`= your project's ID
  * `[image]`= your image name
  * `[tag]`= any string tag of your choice. If unspecified, it defaults to "latest".

In [49]:
import os

PROJECT_ID = "qwiklabs-gcp-04-0ad772141888" # REPLACE WITH YOUR PROJECT NAME

os.environ["PROJECT_ID"] = PROJECT_ID

Let's tag `node-app:0.2`.

In [50]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED              SIZE
node-app                       0.2       b13446ebdebc   About a minute ago   992MB
node-app                       0.1       e0ad8979a21e   7 minutes ago        992MB
node                           latest    7220633f01cd   2 weeks ago          992MB
hello-world                    latest    feb5d9fea6a5   6 weeks ago          13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   8 months ago         1.73GB


**Exercise**

Tag the `node-app:0.2` image with a new image name conforming to the naming convention `gcr.io/[project-id]/[image]:[tag]`. Keep the image and tag names the same.

In [51]:
%%bash
docker tag 'node-app:0.2' 'gcr.io/qwiklabs-gcp-04-0ad772141888/node-app:0.2'

Now when we list our docker images we should see this newly tagged repository.

In [52]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED              SIZE
node-app                                       0.2       b13446ebdebc   About a minute ago   992MB
gcr.io/qwiklabs-gcp-04-0ad772141888/node-app   0.2       b13446ebdebc   About a minute ago   992MB
node-app                                       0.1       e0ad8979a21e   8 minutes ago        992MB
node                                           latest    7220633f01cd   2 weeks ago          992MB
hello-world                                    latest    feb5d9fea6a5   6 weeks ago          13.3kB
gcr.io/inverting-proxy/agent                   <none>    fe507176d0e6   8 months ago         1.73GB


Next, let's push this image to gcr.

**Exercise**

Push this new image to the gcr.

In [54]:
%%bash
docker push 'gcr.io/qwiklabs-gcp-04-0ad772141888/node-app:0.2'

The push refers to repository [gcr.io/qwiklabs-gcp-04-0ad772141888/node-app]
8c48749234d7: Preparing
3fd477af44ee: Preparing
013fc0144002: Preparing
5723726afdfd: Preparing
02784cd376a2: Preparing
3525f11eba01: Preparing
8f56c3340629: Preparing
ba6e5ff31f23: Preparing
9f9f651e9303: Preparing
0b3c02b5d746: Preparing
62a747bf1719: Preparing
3525f11eba01: Waiting
8f56c3340629: Waiting
ba6e5ff31f23: Waiting
9f9f651e9303: Waiting
0b3c02b5d746: Waiting
62a747bf1719: Waiting
5723726afdfd: Layer already exists
02784cd376a2: Layer already exists
013fc0144002: Layer already exists
3525f11eba01: Layer already exists
ba6e5ff31f23: Layer already exists
8f56c3340629: Layer already exists
0b3c02b5d746: Layer already exists
9f9f651e9303: Layer already exists
62a747bf1719: Layer already exists
8c48749234d7: Pushed
3fd477af44ee: Pushed
0.2: digest: sha256:600304b115c5fc216347d64a415222b48a09369990a2038c0a184712d483d7a6 size: 2629


Check that the image exists in `gcr` by visiting the image registry Cloud Console. You can navigate via the console to `Navigation menu > Container Registry` or visit the url from the cell below:

In [55]:
%%bash
echo "http://gcr.io/${PROJECT_ID}/node-app"

http://gcr.io/qwiklabs-gcp-04-0ad772141888/node-app


### Test the published gcr image

Let's test this image. You could start a new VM, ssh into that VM, and install gcloud. For simplicity, we'll just remove all containers and images to simulate a fresh environment.

First, stop and remove all containers using `docker stop` and `docker rm`. **Be careful not to stop the container running this AI Notebook!**.

In [57]:
!docker stop 'my-app' && docker rm 'my-app'

my-app
my-app


In [58]:
!docker stop 'my-app-2' && docker rm 'my-app-2'

my-app-2
my-app-2


Now remove the docker images you've created above using `docker rmi`.

In [59]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED          SIZE
gcr.io/qwiklabs-gcp-04-0ad772141888/node-app   0.2       b13446ebdebc   3 minutes ago    992MB
node-app                                       0.2       b13446ebdebc   3 minutes ago    992MB
node-app                                       0.1       e0ad8979a21e   10 minutes ago   992MB
node                                           latest    7220633f01cd   2 weeks ago      992MB
hello-world                                    latest    feb5d9fea6a5   6 weeks ago      13.3kB
gcr.io/inverting-proxy/agent                   <none>    fe507176d0e6   8 months ago     1.73GB


In [60]:
%%bash
docker rmi node-app:0.2
docker rmi gcr.io/${PROJECT_ID}/node-app:0.2
docker rmi node-app:0.1
docker rmi node:latest
docker rmi -f hello-world:latest

Untagged: node-app:0.2
Untagged: gcr.io/qwiklabs-gcp-04-0ad772141888/node-app:0.2
Untagged: gcr.io/qwiklabs-gcp-04-0ad772141888/node-app@sha256:600304b115c5fc216347d64a415222b48a09369990a2038c0a184712d483d7a6
Deleted: sha256:b13446ebdebc0120106a6bebf312fddfdca638bc2d55b556e83541fea86391c0
Deleted: sha256:5f2e33a18e639f697d9355c9bf7a11ffdbc24d9f40b072657f53a9fcd9b79ccd
Deleted: sha256:d304bd77163850e217a32c090d0e26943b7b0b760d63bc0fdd0ee9f113746fe8
Deleted: sha256:2c16ac1689d6e20f5ab5afe0ff123e27cea31fdbd454cbd2878e84f650a48f2a
Untagged: node-app:0.1
Deleted: sha256:e0ad8979a21e325c0e9b18d19afca809bde0c6d80d371e6425ffe2d67f98834b
Deleted: sha256:a0100313e822b52e941c2338d0ae135b632a32dcbda228c4c90715bff1649363
Deleted: sha256:ce4147ef958b5ee7cc058b85da00a16b94b5e82441100158fb4aa056faff3801
Deleted: sha256:de375618e9800a4c7a807c458f960a5d1f6ec2a15f43f80634ab11dff1dd4959
Deleted: sha256:f1a73ad8418be9491219363bc8c3fa54b31055efbd0a3cb3b6e53ef4221b39f7
Deleted: sha256:ce77970473c888fb78ef1df

Confirm all images are removed with `docker images`.

In [61]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED        SIZE
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   8 months ago   1.73GB


At this point you should have a pseudo-fresh environment. Now, pull the image and run it.

In [62]:
%%bash
docker pull gcr.io/${PROJECT_ID}/node-app:0.2
docker run -p 4000:80 -d gcr.io/${PROJECT_ID}/node-app:0.2

0.2: Pulling from qwiklabs-gcp-04-0ad772141888/node-app
bb7d5a84853b: Pulling fs layer
f02b617c6a8c: Pulling fs layer
d32e17419b7e: Pulling fs layer
c9d2d81226a4: Pulling fs layer
3c24ae8b6604: Pulling fs layer
a7c459a8e2ce: Pulling fs layer
4c986aa83582: Pulling fs layer
8b73f64a6d5d: Pulling fs layer
a50553bfc7b9: Pulling fs layer
7fa0f309c2cd: Pulling fs layer
da473c1b2b4f: Pulling fs layer
3c24ae8b6604: Waiting
8b73f64a6d5d: Waiting
a7c459a8e2ce: Waiting
a50553bfc7b9: Waiting
4c986aa83582: Waiting
7fa0f309c2cd: Waiting
da473c1b2b4f: Waiting
c9d2d81226a4: Waiting
f02b617c6a8c: Verifying Checksum
f02b617c6a8c: Download complete
d32e17419b7e: Verifying Checksum
d32e17419b7e: Download complete
bb7d5a84853b: Verifying Checksum
bb7d5a84853b: Download complete
a7c459a8e2ce: Verifying Checksum
a7c459a8e2ce: Download complete
c9d2d81226a4: Verifying Checksum
c9d2d81226a4: Download complete
8b73f64a6d5d: Verifying Checksum
8b73f64a6d5d: Download complete
a50553bfc7b9: Verifying Checksum
a505

You can check that it's running as expected using before:

In [64]:
!curl http://localhost:4000

Hello World!


Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.